# Append to backtest results and remove duplicates

In [1]:
import pandas as pd

In [2]:
BACKTEST_RESULTS_FILE_PATH = './DOW_30_RL_CNN_seeds_0_etc.csv'

In [3]:
# open existing results as pd.DataFrame
df = pd.read_csv(BACKTEST_RESULTS_FILE_PATH)
print(df.shape)
df.head()

(245, 7)


,gamma_trade,gamma_risk,gamma_hold,seed,excess_return,excess_risk,sharpe
0,0.1,0.1,0.1,0,8.900445,18.115956,0.491304
1,0.1,0.1,1.0,0,8.576749,21.955187,0.390648
2,0.1,0.1,10.0,0,8.736425,17.351895,0.503485
3,0.1,0.1,100.0,0,10.714345,15.713279,0.681866
4,0.1,0.1,1000.0,0,9.923368,15.045008,0.659579


In [4]:
# run backtests

In [5]:
# get new rows to append (with some duplicates)
new_data_dict = {
    'gamma_trade': [0.1, 0.1, 0.1, 0.0],
    'gamma_risk': [0.1, 0.1, 0.1, 0.0],
    'gamma_hold': [0.1, 1.0, 10.0, 0.0],
    'seed': [0, 0, 0, 0],
    'excess_return': [9.99, 9.99, 9.99, 0.0],
    'excess_risk': [9.99, 9.99, 9.99, 0.0],
    'sharpe': [9.99, 9.99, 9.99, 0.0],
    # 'is_pareto': [True, True, True, True]
}

new_df = pd.DataFrame(new_data_dict)
new_df

,gamma_trade,gamma_risk,gamma_hold,seed,excess_return,excess_risk,sharpe
0,0.1,0.1,0.1,0,9.99,9.99,9.99
1,0.1,0.1,1.0,0,9.99,9.99,9.99
2,0.1,0.1,10.0,0,9.99,9.99,9.99
3,0.0,0.0,0.0,0,0.00,0.00,0.00


In [6]:
# append new data and overwrite duplicates if any (for matching gamma-values)

#  find duplicates (per gamma-values)
df = df.set_index(['gamma_trade','gamma_risk','gamma_hold'])
new_df = new_df.set_index(['gamma_trade','gamma_risk','gamma_hold'])
duplicates_df = df.merge(new_df, how='inner', left_index=True, right_index=True)
print(f'duplicates found: {len(duplicates_df)}')

# remove duplicates from original data
df = df[~df.index.isin(duplicates_df.index)]

all_df = pd.concat([df, new_df]).reset_index()
all_df.tail()

duplicates found: 3


,gamma_trade,gamma_risk,gamma_hold,seed,excess_return,excess_risk,sharpe
241,1000.0,100000.0,100000.0,0,2.557553,10.731146,0.23833
242,0.1,0.1,0.1,0,9.990000,9.990000,9.99000
243,0.1,0.1,1.0,0,9.990000,9.990000,9.99000
244,0.1,0.1,10.0,0,9.990000,9.990000,9.99000
245,0.0,0.0,0.0,0,0.000000,0.000000,0.00000


In [10]:
# prepare final results and write to flie
all_df = all_df.sort_values(by=['excess_risk', 'excess_return'])
all_df.to_csv(BACKTEST_RESULTS_FILE_PATH+'_test_.csv', index=False)

all_df

,gamma_trade,gamma_risk,gamma_hold,seed,excess_return,excess_risk,sharpe
245,0.0,0.0,0.0,0,0.000000,0.000000,0.000000
242,0.1,0.1,0.1,0,9.990000,9.990000,9.990000
243,0.1,0.1,1.0,0,9.990000,9.990000,9.990000
244,0.1,0.1,10.0,0,9.990000,9.990000,9.990000
127,10.0,1000.0,1000.0,0,3.114817,10.617843,0.293357
...,...,...,...,...,...,...,...
48,1.0,0.1,10.0,0,9.032600,17.608695,0.512962
4,0.1,1.0,0.1,0,10.264888,17.946579,0.571969
96,10.0,0.1,1.0,0,12.691952,21.359738,0.594200
95,10.0,0.1,0.1,0,11.149700,30.001942,0.371633
